In [1]:
from __future__ import print_function

import numpy as np
import mmd

This follows Dougal's example from: https://github.com/dougalsutherland/mmd/blob/master/examples/mmd%20regression%20example.ipynb

In [6]:
# Generate fake data
n = 500
mean = np.random.normal(0, 10, size=n)
var = np.random.gamma(5, size=n)
n_samp = np.random.randint(10, 500, size=n)
samps = [np.random.normal(m, v, size=s)[:, np.newaxis] for m, v, s in zip(mean, var, n_samp)]

In [40]:
print(len(samps[0]))
print(type(samps[0]))
print(len(samps[1]))
print(type(samps[1]))

112
<type 'numpy.ndarray'>
141
<type 'numpy.ndarray'>


In [47]:
# Get the median pairwise squared distance in the aggregate sample,
# as a heuristic for choosing the bandwidth of the inner RBF kernel.
from sklearn.metrics.pairwise import euclidean_distances
sub = np.vstack(samps)
print(type(sub))
print(sub.shape)
sub = sub[np.random.choice(sub.shape[0], min(1000, sub.shape[0]), replace=False)]
print(sub.shape)
D2 = euclidean_distances(sub, squared=True)
print(D2)
med_2 = np.median(D2[np.triu_indices_from(D2, k=1)], overwrite_input=True)
print(med_2)
del sub, D2

<type 'numpy.ndarray'>
(133846, 1)
(1000, 1)
[[  0.00000000e+00   1.77002081e+02   7.22321143e+02 ...,   1.17240964e+02
    1.11461423e+02   5.24375602e+01]
 [  1.77002081e+02   0.00000000e+00   1.61445215e+03 ...,   5.82353403e+02
    5.69382737e+02   4.22121319e+02]
 [  7.22321143e+02   1.61445215e+03   0.00000000e+00 ...,   2.57546207e+02
    2.66293561e+02   3.85519659e+02]
 ..., 
 [  1.17240964e+02   5.82353403e+02   2.57546207e+02 ...,   0.00000000e+00
    7.30390825e-02   1.28623247e+01]
 [  1.11461423e+02   5.69382737e+02   2.66293561e+02 ...,   7.30390825e-02
    0.00000000e+00   1.09968567e+01]
 [  5.24375602e+01   4.22121319e+02   3.85519659e+02 ...,   1.28623247e+01
    1.09968567e+01   0.00000000e+00]]
127.248788751


In [12]:
from sklearn import cross_validation as cv
from sklearn.kernel_ridge import KernelRidge
import sys

In [13]:
l1_gamma_mults = np.array([1/16, 1/4, 1, 4])
l1_gammas = l1_gamma_mults/med_2 # gamma is inverse of sigma
# since we divide by sigma and the median, we want to gamma by
# the median and multiply by gamma

In [15]:
mmds, mmk_diags = mmd.rbf_mmd(samps, gammas=l1_gammas, squared=True, n_jobs=40, ret_X_diag=True)

In [17]:
# now turn the mmd^2 into a kernel and evaluate the regression for each of the hyperparameter values

# choose parameters for the hyperparameter search
k_fold = list(cv.KFold(n, n_folds=3, shuffle=True))
l2_gamma_mults = np.array([1/4, 1, 4, 8])
alphas = np.array([1/128, 1/64, 1/4, 1, 4])
scores = np.empty((l1_gamma_mults.size, l2_gamma_mults.size, alphas.size, len(k_fold)))
scores.fill(np.nan)

In [22]:
K = np.empty((n, n), dtype=samps[0].dtype)
for l1_gamma_i, l1_gamma in enumerate(l1_gamma_mults * med_2):
    print("l1 gamma {} / {}: {:.4}".format(l1_gamma_i + 1, len(l1_gamma_mults), l1_gamma), file=sys.stderr)
    D2_mmd = mmds[l1_gamma_i]
    
    # get the median of *these* squared distances,
    # to scale the bandwidth of the outer RBF kernel
    mmd_med2 = np.median(D2_mmd[np.triu_indices_from(D2_mmd, k=1)])
    
    for l2_gamma_i, l2_gamma in enumerate(l2_gamma_mults * mmd_med2):
        print("\tl2 gamma {} / {}: {:.4}".format(l2_gamma_i + 1, len(l2_gamma_mults), l2_gamma), file=sys.stderr)
        np.multiply(D2_mmd, -l2_gamma, out=K)
        np.exp(K, out=K)
        
        for alpha_i, alpha in enumerate(alphas):
            ridge = KernelRidge(alpha=alpha, kernel='precomputed')
            these = cv.cross_val_score(ridge, K, mean, cv=k_fold)
            scores[l1_gamma_i, l2_gamma_i, alpha_i, :] = these
            print("\t\talpha {} / {}: {} \t {}".format(alpha_i + 1, len(alphas), alpha, these), file=sys.stderr)

l1 gamma 1 / 4: 0.0
	l2 gamma 1 / 4: 0.0
/home/jenna/anaconda2/lib/python2.7/site-packages/sklearn/linear_model/ridge.py:154: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn("Singular matrix in solving dual problem. Using "
		alpha 1 / 5: 0 	 [-0.31301096 -0.52467986 -0.70612781]
		alpha 2 / 5: 0 	 [-0.31301096 -0.52467986 -0.70612781]
		alpha 3 / 5: 0 	 [-0.31301096 -0.52467986 -0.70612781]
		alpha 4 / 5: 1 	 [-0.01525561 -0.00544371 -0.00179342]
		alpha 5 / 5: 4 	 [-0.01517539 -0.00540549 -0.00177983]
	l2 gamma 2 / 4: 0.0
		alpha 1 / 5: 0 	 [-0.31301096 -0.52467986 -0.70612781]
		alpha 2 / 5: 0 	 [-0.31301096 -0.52467986 -0.70612781]
		alpha 3 / 5: 0 	 [-0.31301096 -0.52467986 -0.70612781]
		alpha 4 / 5: 1 	 [-0.01525561 -0.00544371 -0.00179342]
		alpha 5 / 5: 4 	 [-0.01517539 -0.00540549 -0.00177983]
	l2 gamma 3 / 4: 0.0
		alpha 1 / 5: 0 	 [-0.31301096 -0.52467986 -0.70612781]
		alpha 2 / 5: 0 	 [-0.31301096 -0.52467986 -0.

In [23]:
mean_scores = scores.mean(axis=-1)

In [24]:
import matplotlib.pyplot as plt
import seaborn as sns

/home/jenna/anaconda2/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [25]:
import matplotlib.ticker as ticker

fig = plt.figure(figsize=(8, 15))
for i in range(len(l1_gamma_mults)):
    ax = fig.add_subplot(len(l1_gamma_mults), 1, i+1)
    cax = ax.matshow(mean_scores[i, :, :], vmin=0, vmax=1)
    
    ax.set_yticklabels([''] + ['{:.3}'.format(g * mmd_med2) for g in l2_gamma_mults])
    ax.set_xticklabels([''] + ['{:.3}'.format(a) for a in alphas])
    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))
    
    fig.colorbar(cax)
    ax.set_title("L1 gamma: {}".format(l1_gamma_mults[i] * med_2))
plt.tight_layout()

ValueError: Precision not allowed in integer format specifier

In [26]:
best_l1_gamma_i, best_l2_gamma_i, best_alpha_i = np.unravel_index(mean_scores.argmax(), mean_scores.shape)
best_l1_gamma = l1_gamma_mults[best_l1_gamma_i] * med_2
best_l2_gamma = l2_gamma_mults[best_l2_gamma_i] * mmd_med2
best_alpha = alphas[best_alpha_i]

In [27]:
best_l1_gamma_i, best_l2_gamma_i, best_alpha_i


(2, 1, 3)

In [28]:
# now train a model on the full training set
# get the training kernel
D2_mmd = mmds[best_l1_gamma_i]
np.multiply(D2_mmd, -best_l2_gamma, out=K)
np.exp(K, out=K)

ridge = KernelRidge(alpha=best_alpha, kernel='precomputed')
ridge.fit(K, mean)

KernelRidge(alpha=1, coef0=1, degree=3, gamma=None, kernel='precomputed',
      kernel_params=None)

In [29]:
# evaluate on new data
# generate some test data from the same distribution
t_n = 100
t_mean = np.random.normal(0, 10, size=t_n)
t_var = np.random.gamma(5, size=t_n)
t_n_samp = np.random.randint(10, 5000, size=t_n)
t_samps = [np.random.normal(m, v, size=s)[:, np.newaxis] for m, v, x, in zip(t_mean, t_var, t_n_samp)]

In [30]:
# apply the kernel generated using the training data to the test data
t_K = mmd.rbf_mmd(t_samps, samps, gammas=best_l1_gamma, squared=True,
                 Y_diag=mmk_diags[best_l1_gamma_i], n_jobs=20)
t_K *= -best_l2_gamma
np.exp(t_K, out=t_K)

array([[ 0.73170252,  0.66805928,  0.73492671, ...,  0.64948179,
         0.66651361,  0.63484761],
       [ 0.73384386,  0.67205118,  0.73875124, ...,  0.65359307,
         0.67040808,  0.6326097 ],
       [ 0.73258752,  0.66926065,  0.73610474, ...,  0.65055619,
         0.66783187,  0.63617991],
       ..., 
       [ 0.73062304,  0.66783913,  0.73462421, ...,  0.64914168,
         0.66578466,  0.63335361],
       [ 0.73427434,  0.66786371,  0.73659583, ...,  0.64762565,
         0.67034907,  0.62948137],
       [ 0.73659605,  0.66723926,  0.73655894, ...,  0.64780669,
         0.66850917,  0.63021556]])

In [35]:
preds = ridge.predict(t_K)
plt.figure(figsize=(5,5))
plt.scatter(t_mean, preds)